## Step 1: 환경 설정 및 필요한 라이브러리 설치

In [67]:
!pip install datasets openai


## Step 2: OpenAI API 키 설정

In [105]:
from openai import OpenAI
from google.colab import userdata

# OpenAI API 키 설정
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

## Step 3: 데이터셋 불러오기

In [106]:
import json
import requests

# GitHub에서 데이터셋 불러오기
url = "https://raw.githubusercontent.com/NomaDamas/KICE_slayer_AI_Korean/master/data/2023_11_KICE.json"
response = requests.get(url)
data = json.loads(response.text)

print(data[0])  # 첫 번째 문제 출력
print(data[1])  # 두 번째 문제 출력
print(data[2])  # 세 번째 문제 출력

# 총 ID와 문제 수 출력
total_ids = len(data)
total_problems = sum(len(item['problems']) for item in data)
print(f"\n총 {total_ids}개의 ID에서 {total_problems}개의 문제가 있습니다.")


{'id': '2023_11_KICE_1-3', 'paragraph': '사람들이 지속적으로 책을 읽는 이유 중 하나는 즐거움이다. 독서의 즐거움에는 여러 가지가 있겠지만 그 중심에는 ‘소통의 즐거움’이 있다.독자는 독서를 통해 책과 소통하는 즐거움을 경험한다. 독서는필자와 간접적으로 대화하는 소통 행위이다. 독자는 자신이 속한사회나 시대의 영향 아래 필자가 속해 있거나 드러내고자 하는 사회나 시대를 경험한다. 직접 경험하지 못했던 다양한 삶을 필자를 매개로 만나고 이해하면서 독자는 더 넓은 시야로 세계를바라볼 수 있다. 이때 같은 책을 읽은 독자라도 독자의 배경지식이나 관점 등의 독자 요인, 읽기 환경이나 과제 등의 상황 요인이 다르므로, 필자가 보여 주는 세계를 그대로 수용하지 않고 저마다 소통 과정에서 다른 의미를 구성할 수 있다.[A] (이러한 소통은 독자가 책의 내용에 대해 질문하고 답을 찾아내는 과정에서 가능해진다. 독자는 책에서 답을 찾는 질문, 독자 자신에게서 답을 찾는 질문 등을 제기할 수 있다. 전자의 경우 책에 명시된 내용에서 답을 발견할 수 있고, 책의 내용들을 관계 지으며 답에 해당하는 내용을 스스로 구성할 수도 있다. 또한 후자의 경우 책에는 없는 독자의 경험에서 답을 찾을 수 있다. 이런 질문들을 풍부히 생성하고 주체적으로 답을 찾을 때 소통의 즐거움은 더 커진다.)한편 독자는 ㉠ (다른 독자와 소통하는 즐거움을 경험할 수도 있다.) 책과의 소통을 통해 개인적으로 형성한 의미를 독서 모임이나 독서 동아리 등에서 다른 독자들과 나누는 일이 이에 해당한다. 비슷한 해석에 서로 공감하며 기존 인식을 강화하거나 관점의 차이를 확인하고 기존 인식을 조정하는 과정에서, 독자는자신의 인식을 심화 확장할 수 있다. 최근 소통 공간이 온라인으로 확대되면서 독서를 통해 다른 독자들과 소통하며 즐거움을누리는 양상이 더 다양해지고 있다. 자신의 독서 경험을 담은 글이나 동영상을 생산 공유함으로써, 책을 읽지 않은 타인이 책과 소통하도록 돕는 것도 책을 통

## Step 4:예측 함수 (GPT-4o 모델 사용)

In [130]:
def prediction(paragraph, problem):
    # 문제, 선택지를 추출
    question_text = problem['question']  # 문제 텍스트 추출
    choices = problem['choices']  # 선택지 추출
    actual_answer_index = problem['answer'] - 1  # 실제 정답 인덱스를 0 기반으로 변환

    # 선택지를 보기 형식으로 포맷팅 (1부터 시작하는 숫자로 출력)
    choices_text = "\n".join([f"{i+1}. {choice}" for i, choice in enumerate(choices)])

    # GPT-4o에게 보낼 질문 형식 지정
    user_prompt = f"""
    다음 지문을 읽고 질문에 답변해 주세요. 정답을 선택지 번호로만 답해 주세요.

    지문: {paragraph}

    질문: {question_text}

    선택지:
    {choices_text}

    답변은 숫자만 입력해 주세요.
    """

    # GPT-4o API로 질문을 보냄
    message = [{"role": "user", "content": user_prompt}]
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=message,
        n=1,  # 1개의 응답만 생성
        max_tokens=4096,
        temperature=0.5,  # 답변의 다양성 제어
        frequency_penalty=0.0  # 동일한 단어 반복 방지
    )

    # 첫 번째 응답에서 생성된 답변 추출
    generated_answer = response.choices[0].message.content.strip()

    # 예측된 답변의 기본값 설정 (숫자가 아닐 경우 대비)
    predicted_answer_index = None

    # GPT-4o가 숫자를 반환했는지 확인
    if generated_answer.isdigit():
        predicted_answer_index = int(generated_answer) - 1  # 인덱스를 0부터 시작하게 변환
        predicted_answer_text = choices[predicted_answer_index]  # 선택지 내용 추출
    else:
        predicted_answer_text = generated_answer  # 숫자가 아닐 경우 그대로 사용

    # 실제 정답 내용 추출
    actual_answer_text = choices[actual_answer_index]

    # 예측된 답변과 실제 정답 비교
    is_correct = predicted_answer_index == actual_answer_index if predicted_answer_index is not None else False

    # 결과 출력
    print(f"예측된 답변 ({predicted_answer_index + 1 if predicted_answer_index is not None else 'None'}): {predicted_answer_text}")
    print(f"실제 정답 ({actual_answer_index + 1}): {actual_answer_text}")

    return f"정답 여부: {is_correct}", is_correct

## Step 5: 모든 문제 평가하기

In [132]:
def evaluate_all_problems(data):
    correct_count = 0  # 맞은 문제 수
    total_problems = 0  # 총 문제 수

    # 전체 ID에서 문제를 반복
    for item in data:
        paragraph = item['paragraph']  # 각 ID의 paragraph 추출
        for problem in item['problems']:
            total_problems += 1  # 총 문제 수 증가
            result, is_correct = prediction(paragraph, problem)

            # 예측 결과가 True면 맞은 문제로 카운팅
            if is_correct:
                correct_count += 1

    # 전체 문제에 대한 정확도 계산
    accuracy = (correct_count / total_problems) * 100

    # 결과 출력
    print(f"총 {total_problems}개의 문제 중 {correct_count}개 맞았습니다.")
    print(f"정확도: {accuracy:.2f}%")

    # 50점 이상이면 통과
    if accuracy >= 50:
        print("합격!")
    else:
        print("불합격, 다시 도전하세요.")

# 전체 문제 풀기
evaluate_all_problems(data)

예측된 답변 (4): 독자의 배경지식, 관점, 읽기 환경, 과제는 독자의 의미 구성에 영향을 주는 독자 요인이다.
실제 정답 (4): 독자의 배경지식, 관점, 읽기 환경, 과제는 독자의 의미 구성에 영향을 주는 독자 요인이다.
예측된 답변 (4): ⓑ에는 ‘책에서 답을 찾는 질문’이, ⓒ에는 그에 대한 답을 ‘책의 내용들을 관계 지으며’ 찾아내는 모습이 나타난다.
실제 정답 (5): ⓓ에는 ‘독자 자신에게서 답을 찾는 질문’이, ⓔ에는 그에 대한 답을 ‘독자의 경험’에서 찾아내는 모습이 나타난다.
예측된 답변 (1): 스스로 독서 계획을 세우고 자신에게 필요한 책을 찾아 개인적으로 읽는 과정에서 경험할 수 있겠군.
실제 정답 (1): 스스로 독서 계획을 세우고 자신에게 필요한 책을 찾아 개인적으로 읽는 과정에서 경험할 수 있겠군.
예측된 답변 (4): (가)는 유서의 특성과 의의를 설명하였고, (나)는 유서 편찬에서 특정 학문의 수용 양상을 시기별로 소개하였다.
실제 정답 (4): (가)는 유서의 특성과 의의를 설명하였고, (나)는 유서 편찬에서 특정 학문의 수용 양상을 시기별로 소개하였다.
예측된 답변 (5): 중국에서는 주로 서적에서 발췌한 내용을 비교하고 해석을 덧붙여 유서를 편찬하였다.
실제 정답 (5): 중국에서는 주로 서적에서 발췌한 내용을 비교하고 해석을 덧붙여 유서를 편찬하였다.
예측된 답변 (3): 평가를 더하는 저술로서 ㉮의 성격은, ㉡에서 중국 학문의 진보성을 확인하고자 서학을 활용한 것에서 나타난다.
실제 정답 (3): 평가를 더하는 저술로서 ㉮의 성격은, ㉡에서 중국 학문의 진보성을 확인하고자 서학을 활용한 것에서 나타난다.
예측된 답변 (2): 주자학에 매몰되어 세상의 여러 이치를 연구하지 않는 것은 널리 배우고 익히는 앎의 바른 방법이 아닐 것이다.
실제 정답 (2): 주자학에 매몰되어 세상의 여러 이치를 연구하지 않는 것은 널리 배우고 익히는 앎의 바른 방법이 아닐 것이다.
예측된 답변 (3): 당대 지식을 망라하고 서양 관련 지식

모든 문제를 처리한 결과, 총 45개의 문제 중 37개를 맞추어 **82.22%**의 정확도를 기록했습니다. 이로써 목표 기준인 50%를 넘어 합격하였습니다.